# Bayesian Network Example with `pgmpy` and `pomegranate`

From the [tutorial](https://pgmpy.org/detailed_notebooks/2.%20Bayesian%20Networks.html), we want to instantiate the network as follows:

![fig1](fig1.png)

The above 'student' model describes how grades would be impacted by students intelligence, test difficulty, test grade, and letter grade for the course.

>In pgmpy we define the network structure and the CPDs (conditional probability distributions) separately and then associate them with the structure. Here’s an example for defining the above model:

* variable_card = variable cardinality, i.e. the number of states the variable can take

In [79]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD

# Defining the model structure. We can define the network by just passing a list of edges.
model = BayesianNetwork([('D', 'G'), ('I', 'G'), ('G', 'L'), ('I', 'S')])

In [80]:
# CPDs can also be defined using the state names of the variables. If the state names are not provided
# like in the previous example, pgmpy will automatically assign names as: 0, 1, 2, ....

cpd_d_sn = TabularCPD(variable='D', variable_card=2, values=[[0.6], [0.4]], state_names={'D': ['Easy', 'Hard']})
cpd_i_sn = TabularCPD(variable='I', variable_card=2, values=[[0.7], [0.3]], state_names={'I': ['Dumb', 'Intelligent']})
cpd_g_sn = TabularCPD(variable='G', variable_card=3,
                      values=[[0.3, 0.05, 0.9,  0.5],
                              [0.4, 0.25, 0.08, 0.3],
                              [0.3, 0.7,  0.02, 0.2]],
                      evidence=['I', 'D'],
                      evidence_card=[2, 2],
                      state_names={'G': ['A', 'B', 'C'],
                                   'I': ['Dumb', 'Intelligent'],
                                   'D': ['Easy', 'Hard']})

cpd_l_sn = TabularCPD(variable='L', variable_card=2,
                      values=[[0.1, 0.4, 0.99],
                              [0.9, 0.6, 0.01]],
                      evidence=['G'],
                      evidence_card=[3],
                      state_names={'L': ['Bad', 'Good'],
                                   'G': ['A', 'B', 'C']})

cpd_s_sn = TabularCPD(variable='S', variable_card=2,
                      values=[[0.95, 0.2],
                              [0.05, 0.8]],
                      evidence=['I'],
                      evidence_card=[2],
                      state_names={'S': ['Bad', 'Good'],
                                   'I': ['Dumb', 'Intelligent']})

# These defined CPDs can be added to the model. Since, the model already has CPDs associated to variables, it will
# show warning that pmgpy is now replacing those CPDs with the new ones.
model.add_cpds(cpd_d_sn, cpd_i_sn, cpd_g_sn, cpd_l_sn, cpd_s_sn)
model.check_model()

True

In [81]:
# We can now call some methods on the BayesianModel object.
model.get_cpds()

[<TabularCPD representing P(D:2) at 0x7fcd2d6798b0>,
 <TabularCPD representing P(I:2) at 0x7fcd2d679850>,
 <TabularCPD representing P(G:3 | I:2, D:2) at 0x7fcd2d6798e0>,
 <TabularCPD representing P(L:2 | G:3) at 0x7fcd2d679880>,
 <TabularCPD representing P(S:2 | I:2) at 0x7fcd2d679910>]

In [83]:
# Printing a CPD with it's state names defined.
print(model.get_cpds('G'))

+------+---------+---------+----------------+----------------+
| I    | I(Dumb) | I(Dumb) | I(Intelligent) | I(Intelligent) |
+------+---------+---------+----------------+----------------+
| D    | D(Easy) | D(Hard) | D(Easy)        | D(Hard)        |
+------+---------+---------+----------------+----------------+
| G(A) | 0.3     | 0.05    | 0.9            | 0.5            |
+------+---------+---------+----------------+----------------+
| G(B) | 0.4     | 0.25    | 0.08           | 0.3            |
+------+---------+---------+----------------+----------------+
| G(C) | 0.3     | 0.7     | 0.02           | 0.2            |
+------+---------+---------+----------------+----------------+


In [84]:
# Getting all the local independencies in the network.
model.local_independencies(['D', 'I', 'S', 'G', 'L'])

(D ⟂ S, I)
(I ⟂ D)
(S ⟂ L, D, G | I)
(G ⟂ S | D, I)
(L ⟂ S, D, I | G)

## Variable Elimination
We know that:

$ P(D, I, G, L, S) = P(L|G) * P(S|I) * P(G|D, I) * P(D) * P(I) $

Now let’s say we just want to compute the probability of G. For that we will need to marginalize over all the other variables.

Now since not all the conditional distributions depend on all the variables we can push the summations inside:

$ P(G) = \sum_D \sum_I \sum_L \sum_S P(L|G) * P(S|I) * P(G|D, I) * P(D) * P(I) $

$ P(G) = \sum_D P(D) \sum_I P(G|D, I) * P(I) \sum_S P(S|I) \sum_L P(L|G) $

So, by pushing the summations inside we have saved a lot of computation because we have to now iterate over much smaller tables.

Let’s take an example for inference using Variable Elimination in pgmpy:

In [85]:
from pgmpy.inference import VariableElimination
infer = VariableElimination(model)
g_dist = infer.query(['G'])
print(g_dist)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+------+----------+
| G    |   phi(G) |
+======+==========+
| G(A) |   0.3620 |
+------+----------+
| G(B) |   0.2884 |
+------+----------+
| G(C) |   0.3496 |
+------+----------+


There can be cases in which we want to compute the conditional distribution let’s say $ P(G | D=0, I=1) $. In such cases we need to modify our equations a bit:

$ P(G | D=0, I=1) = \sum_L \sum_S P(L|G) * P(S| I=1) * P(G| D=0, I=1) * P(D=0) * P(I=1) $ $ P(G | D=0, I=1) = P(D=0) * P(I=1) * P(G | D=0, I=1) * \sum_L P(L | G) * \sum_S P(S | I=1) $

In pgmpy we will just need to pass an extra argument in the case of conditional distributions:

In [86]:
print(infer.query(['G'], evidence={'D': 'Easy', 'I': 'Intelligent'}))

0it [00:00, ?it/s]

0it [00:00, ?it/s]

+------+----------+
| G    |   phi(G) |
+======+==========+
| G(A) |   0.9000 |
+------+----------+
| G(B) |   0.0800 |
+------+----------+
| G(C) |   0.0200 |
+------+----------+


## Example 2: 

![fig2.png](fig2.png)

In [88]:
model = BayesianNetwork([('T', 'J'), ('T', 'K'), ('J', 'M'), ('K', 'M')])

cpd_t = TabularCPD(variable='T', variable_card=2, values=[[0.4], [0.6]], state_names={'T':['low', 'high']})
cpd_j = TabularCPD(variable='J', variable_card=2, 
                   values=[[0.5, 0.7], 
                           [0.5, 0.3]], 
                   evidence=['T'],
                   evidence_card=[2],
                   state_names={'J':['yes', 'no'],
                               'T':['low', 'high']})
cpd_k = TabularCPD(variable='K', variable_card=2, 
                   values=[[0.4, 0.75], 
                           [0.6, 0.25]], 
                   evidence=['T'],
                   evidence_card=[2],
                   state_names={'K':['yes', 'no'],
                               'T':['low', 'high']})
cpd_m = TabularCPD(variable='M', variable_card = 2,
                  values=[[.5, 0, 0, 0],
                         [.5, 1, 1, 1]],
                  evidence=['J', 'K'],
                  evidence_card=[2, 2],
                  state_names={'M': ['yes', 'no'],
                              'J': ['yes', 'no'],
                              'K': ['yes', 'no']})

# These defined CPDs can be added to the model. Since, the model already has CPDs associated to variables, it will
# show warning that pmgpy is now replacing those CPDs with the new ones.
model.add_cpds(cpd_t, cpd_j, cpd_k, cpd_m)
# model.add_cpds(cpd_d_sn, cpd_i_sn, cpd_g_sn, cpd_l_sn, cpd_s_sn)
model.check_model()

True

In [89]:
# Printing a CPD with it's state names defined.
print(model.get_cpds('J'))

+--------+--------+---------+
| T      | T(low) | T(high) |
+--------+--------+---------+
| J(yes) | 0.5    | 0.7     |
+--------+--------+---------+
| J(no)  | 0.5    | 0.3     |
+--------+--------+---------+


### Inference

What is the probability that they meet $P(M)=?$

In [90]:
infer = VariableElimination(model)
g_dist = infer.query(['M'])
print(g_dist)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

+--------+----------+
| M      |   phi(M) |
+========+==========+
| M(yes) |   0.1975 |
+--------+----------+
| M(no)  |   0.8025 |
+--------+----------+


What's the probability that John goes for a run, given that they don't meet?

i.e.: $P(J|M=\text{no})$

In [91]:
print(infer.query(['J'], evidence={'M': 'no'}))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+--------+----------+
| J      |   phi(J) |
+========+==========+
| J(yes) |   0.5265 |
+--------+----------+
| J(no)  |   0.4735 |
+--------+----------+


What is $P(M=\text{yes}|T=\text{high})?$

In [92]:
print(infer.query(['M'], evidence={'T': 'high'}))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+--------+----------+
| M      |   phi(M) |
+========+==========+
| M(yes) |   0.2625 |
+--------+----------+
| M(no)  |   0.7375 |
+--------+----------+


What is $P(J|K)?$

In [93]:
print(infer.query(['J'], evidence={'K': 'yes'}))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

+--------+----------+
| J      |   phi(J) |
+========+==========+
| J(yes) |   0.6475 |
+--------+----------+
| J(no)  |   0.3525 |
+--------+----------+


## Example 3

![fig3.png](fig3.png)

In [94]:
model = BayesianNetwork([('T', 'N'), ('R', 'N'), ('N', 'U'), ('U', 'P'), ('U', 'C'), ('P', 'C')])

T = TabularCPD(variable='T', variable_card=2, values=[[0.25], [0.75]], state_names={'T':['Hospital', 'DO']})
R = TabularCPD(variable='R', variable_card=2, values=[[0.55], [0.45]], state_names={'R':['North', 'South']})
N = TabularCPD(variable='N', variable_card=2,
                  values=[[.9, .75, .8, .6],
                         [.1, .25, .2, .4]],
                  evidence=['T', 'R'],
                  evidence_card=[2, 2],
                  state_names={'N': ['1', '≥2'],
                              'T': ['Hospital', 'DO'],
                              'R': ['North', 'South']})
U = TabularCPD(variable='U', variable_card=2,
              values=[[.5, .3],
                     [.5, .7]],
              evidence=['N'],
              evidence_card=[2],
              state_names={'U': ['low', 'high'],
                          'N': ['1', '≥2']})
P = TabularCPD(variable='P', variable_card=2,
              values=[[.3, .6],
                     [.7, .4]],
              evidence=['U'],
              evidence_card=[2],
              state_names={'P': ['yes', 'no'],
                          'U': ['low', 'high']})
C = TabularCPD(variable='C', variable_card=2,
                  values=[[.4, .2, .25, .1],
                         [.6, .8, .75, .9]],
                  evidence=['U', 'P'],
                  evidence_card=[2, 2],
                  state_names={'C': ['yes', 'no'],
                              'U': ['low', 'high'],
                              'P': ['yes', 'no']})

model.add_cpds(T, R, N, U, P, C)
model.check_model()

True

In [95]:
print(model.get_cpds('C'))

+--------+--------+--------+---------+---------+
| U      | U(low) | U(low) | U(high) | U(high) |
+--------+--------+--------+---------+---------+
| P      | P(yes) | P(no)  | P(yes)  | P(no)   |
+--------+--------+--------+---------+---------+
| C(yes) | 0.4    | 0.2    | 0.25    | 0.1     |
+--------+--------+--------+---------+---------+
| C(no)  | 0.6    | 0.8    | 0.75    | 0.9     |
+--------+--------+--------+---------+---------+


### Independence

Is customer cancellation independent of customer type?

In [96]:
# Getting all the local independencies in the network.
model.local_independencies(['C', 'T'])

(C ⟂ R, T, N | P, U)
(T ⟂ R)

Type is not independent of cancellation

Next, is customer cancellation independent of customer type given usage?

Type is indeed conditionally independent of customer cancellation given usage

### Inference

In [97]:
infer = VariableElimination(model)

Find probability that a customer cancels given that it's in the north region:

In [98]:
print(infer.query(['C'], evidence={'R': 'North'}))

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

+--------+----------+
| C      |   phi(C) |
+========+==========+
| C(yes) |   0.2226 |
+--------+----------+
| C(no)  |   0.7774 |
+--------+----------+


Find the probability that a customer is a hospital given that it is cancelled:

In [99]:
print(infer.query(['T'], evidence={'C': 'yes'}))

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

+-------------+----------+
| T           |   phi(T) |
+=============+==========+
| T(Hospital) |   0.2515 |
+-------------+----------+
| T(DO)       |   0.7485 |
+-------------+----------+


# Pomegranate

In [100]:
from pomegranate import *

guest = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})
prize = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})
monty = ConditionalProbabilityTable(
        [['A', 'A', 'A', 0.0],
         ['A', 'A', 'B', 0.5],
         ['A', 'A', 'C', 0.5],
         ['A', 'B', 'A', 0.0],
         ['A', 'B', 'B', 0.0],
         ['A', 'B', 'C', 1.0],
         ['A', 'C', 'A', 0.0],
         ['A', 'C', 'B', 1.0],
         ['A', 'C', 'C', 0.0],
         ['B', 'A', 'A', 0.0],
         ['B', 'A', 'B', 0.0],
         ['B', 'A', 'C', 1.0],
         ['B', 'B', 'A', 0.5],
         ['B', 'B', 'B', 0.0],
         ['B', 'B', 'C', 0.5],
         ['B', 'C', 'A', 1.0],
         ['B', 'C', 'B', 0.0],
         ['B', 'C', 'C', 0.0],
         ['C', 'A', 'A', 0.0],
         ['C', 'A', 'B', 1.0],
         ['C', 'A', 'C', 0.0],
         ['C', 'B', 'A', 1.0],
         ['C', 'B', 'B', 0.0],
         ['C', 'B', 'C', 0.0],
         ['C', 'C', 'A', 0.5],
         ['C', 'C', 'B', 0.5],
         ['C', 'C', 'C', 0.0]], [guest, prize])

s1 = Node(guest, name="guest")
s2 = Node(prize, name="prize")
s3 = Node(monty, name="monty")

model = BayesianNetwork("Monty Hall Problem")
model.add_states(s1, s2, s3)
model.add_edge(s1, s3)
model.add_edge(s2, s3)
model.bake()